# Homework Lab 2: Text Preprocessing with Vietnamese
**Overview:** In this exercise, we will build a text preprocessing program for Vietnamese.

Import the necessary libraries. Note that we are using the underthesea library for Vietnamese tokenization. To install it, follow the instructions below. ([link](https://github.com/undertheseanlp/underthesea))

In [1]:
%pip install tqdm
%pip install maturin
%pip install --upgrade pip setuptools wheel
%pip install underthesea --no-build-isolation

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, glob
import codecs
import sys
import re
from tqdm import tqdm
from underthesea import word_tokenize

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- I do this exercise on my visual studio code
- So i need to clone the 2 repo to my working directory to use them
- Then for the VNTC repo, I need to extract manually by hand these 2 files
  - Lab02\VNTC\Data\10Topics\Ver1.1\Test_Full.rar
  - Lab02\VNTC\Data\10Topics\Ver1.1\Train_Full.rar
- Here is the relative path from the working directory to the files that will be used in this exercise after cloning the 2 repo and extract 
  - Lab02\VNTC\Data\10Topics\Ver1.1\Test_Full
  - Lab02\VNTC\Data\10Topics\Ver1.1\Train_Full
  - Lab02\vietnamese-stopwords\vietnamese-stopwords.txt

In [3]:
!git clone https://github.com/duyvuleo/VNTC.git
!git clone https://github.com/stopwords/vietnamese-stopwords

fatal: destination path 'VNTC' already exists and is not an empty directory.
fatal: destination path 'vietnamese-stopwords' already exists and is not an empty directory.


In [4]:
import sys
import os
import platform

# Python environment details
print("Python executable being used:", sys.executable)
print("Python version:", sys.version)

# Operating System details
print("Operating System:", platform.system())
print("OS Version:", platform.version())
print("OS Release:", platform.release())

# Machine and architecture details
print("Machine:", platform.machine())

# Visual Studio Code details (based on environment variable)
vscode_info = os.environ.get('VSCODE_PID', None)
if vscode_info:
    print("Running in Visual Studio Code")
else:
    print("Not running in Visual Studio Code")

Python executable being used: c:\Users\User\AppData\Local\Programs\Python\Python311\python.exe
Python version: 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]
Operating System: Windows
OS Version: 10.0.19045
OS Release: 10
Machine: AMD64
Running in Visual Studio Code


## Question 1: Create a Corpus and Survey the Data

The data in this section is partially extracted from the [VNTC](https://github.com/duyvuleo/VNTC) dataset. VNTC is a Vietnamese news dataset covering various topics. In this section, we will only process the science topic from VNTC. We will create a corpus from both the train and test directories. Complete the following program:

- Write `sentences_list` to a file named `dataset_name.txt`, with each element as a document on a separate line.
- Check how many documents are in the corpus.


In [5]:
dataset_name = "VNTC_khoahoc"
# path = [
#     r"E:\2_LEARNING_BKU\2_File_2\K22_HK242\CO3085_NLP\BT\Lab02\VNTC\Data\10Topics\Ver1.1\Train_Full",
#     r"E:\2_LEARNING_BKU\2_File_2\K22_HK242\CO3085_NLP\BT\Lab02\VNTC\Data\10Topics\Ver1.1\Test_Full"
# ]
# Use relative paths
path = [
    os.path.join(os.getcwd(), "VNTC", "Data", "10Topics", "Ver1.1", "Train_Full"),
    os.path.join(os.getcwd(), "VNTC", "Data", "10Topics", "Ver1.1", "Test_Full")
]

# Debug: Print the contents of each directory
print("Training directory contents:", os.listdir(path[0]))
print("Testing directory contents:", os.listdir(path[1]))

if os.listdir(path[0]) == os.listdir(path[1]):
    folder_list = [os.listdir(path[0]), os.listdir(path[1])]
    print("Train labels = Test labels")
    print("Folder list (train/test):", folder_list)
else:
    print("Train labels differ from test labels")
    folder_list = []
    print("Folder list is empty:", folder_list)

doc_num = 0
sentences_list = []

for i in range(2):
    for folder_name in folder_list[i]:
        if folder_name != "Khoa hoc": 
            continue
        
        folder_path = path[i] + "\\" + folder_name
        print("Processing folder:", folder_path)
        
        # Use tqdm for file processing progress
        for file_name in tqdm(glob.glob(os.path.join(folder_path, "*.txt")), desc=f"Processing {folder_name}", unit="file"):
            with codecs.open(file_name, "r", encoding="utf-16") as f:
                file_content = f.read().replace("\r\n", " ").strip()
                sentences_list.append(file_content)
            
            doc_num += 1

#### YOUR CODE HERE ####
# Write `sentences_list` to a file
output_file = f"{dataset_name}.txt"
with open(output_file, "w", encoding="utf-8") as out_file:
    for sentence in sentences_list:
        out_file.write(sentence + "\n")

# Display the number of documents in the corpus
print(f"Number of documents in the corpus: {doc_num}")
print(f"Corpus written to: {output_file}")
#### END YOUR CODE #####

Training directory contents: ['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh', 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa', 'Vi tinh']
Testing directory contents: ['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh', 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa', 'Vi tinh']
Train labels = Test labels
Folder list (train/test): [['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh', 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa', 'Vi tinh'], ['Chinh tri Xa hoi', 'Doi song', 'Khoa hoc', 'Kinh doanh', 'Phap luat', 'Suc khoe', 'The gioi', 'The thao', 'Van hoa', 'Vi tinh']]
Processing folder: e:\2_LEARNING_BKU\2_File_2\K22_HK242\CO3085_NLP\BT\Lab02\VNTC\Data\10Topics\Ver1.1\Train_Full\Khoa hoc


Processing Khoa hoc: 100%|██████████| 1820/1820 [00:05<00:00, 305.11file/s]


Processing folder: e:\2_LEARNING_BKU\2_File_2\K22_HK242\CO3085_NLP\BT\Lab02\VNTC\Data\10Topics\Ver1.1\Test_Full\Khoa hoc


Processing Khoa hoc: 100%|██████████| 2096/2096 [00:06<00:00, 332.88file/s]


Number of documents in the corpus: 3916
Corpus written to: VNTC_khoahoc.txt


## Question 2: Write Preprocessing Functions







### Question 2.1: Write a Function to Clean Text
Hint:
- The text should only retain the following characters: aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0-9(),!?\'\
- Then trim the whitespace in the input text.

In [6]:
def clean_str(string):
    #### YOUR CODE HERE ####
    
    # Define the allowed characters (including Vietnamese-specific ones)
    allowed_chars = "aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0-9(),!?\\'"
    
    # Create a regex pattern that matches all allowed characters
    pattern = f"[^{allowed_chars}]"  # Matches anything NOT in the allowed set

    # Remove all unwanted characters
    cleaned_string = re.sub(pattern, " ", string)

    # Trim excessive whitespace
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string).strip()

    return cleaned_string

    #### END YOUR CODE #####

### Question 2.2: Write a Function to Convert Text to Lowercase

In [7]:
# make all text lowercase
def text_lowercase(string):
    #### YOUR CODE HERE ####
    return string.lower()
    #### END YOUR CODE #####

### Question 2.3: Tokenize Words
Hint: Use the `word_tokenize()` function imported above with two parameters: `strings` and `format="text"`.


In [8]:
def tokenize(strings):
    #### YOUR CODE HERE ####
    tokens = word_tokenize(strings, format="text")
    return tokens
    #### END YOUR CODE #####

### Question 2.4: Remove Stop Words
To remove stop words, we use a list of Vietnamese stop words stored in the file `./vietnamese-stopwords.txt`. Complete the following program:
- Check each word in the text (`strings`). If a word is not in the stop words list, add it to `doc_words`.


In [9]:
# relative to the vn stopwords dataset: Lab02\vietnamese-stopwords\vietnamese-stopwords.txt
def remove_stopwords(strings):
    # Define the path to the Vietnamese stop words file (relative path)
    stopwords_path = os.path.join(os.getcwd(), "vietnamese-stopwords", "vietnamese-stopwords.txt")
    
    with open(stopwords_path, "r", encoding="utf-8") as f:
        stop_words = set(f.read().splitlines())
    
    words = strings.split()
    doc_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(doc_words)

## Question 2.5: Build a Preprocessing Function
Hint: Call the functions `clean_str`, `text_lowercase`, `tokenize`, and `remove_stopwords` in order, then return the result from the function.


In [10]:
def text_preprocessing(strings):
    #### YOUR CODE HERE ####
    
    # Step 1: Clean the string to remove unwanted characters
    cleaned_text = clean_str(strings)
    
    # Step 2: Convert the text to lowercase
    lowercase_text = text_lowercase(cleaned_text)
    
    # Step 3: Tokenize the text into words
    tokenized_text = tokenize(lowercase_text)
    
    # Step 4: Remove stopwords from the tokenized text
    processed_text = remove_stopwords(tokenized_text)
    
    return processed_text
    
    #### END YOUR CODE #####

## Question 3: Perform Preprocessing
Now, we will read the corpus from the file created in Question 1. After that, we will call the preprocessing function for each document in the corpus.

Hint: Call the `text_preprocessing()` function with `doc_content` as the input parameter and save the result in the variable `temp1`.


In [11]:
#### YOUR CODE HERE ####
clean_docs = []
corpus_file = "VNTC_khoahoc.txt"

# Precompute total lines
with open(corpus_file, "r", encoding="utf-8") as file:
    total_lines = sum(1 for _ in file)

# Open the corpus file and preprocess each document with a progress bar
with open(corpus_file, "r", encoding="utf-8") as file:
    for line in tqdm(file, desc="Processing documents", unit="doc", total=total_lines):
        temp1 = text_preprocessing(line.strip())
        clean_docs.append(temp1)

print("\nLength of clean_docs = ", len(clean_docs))
print("clean_docs[0]:\n" + clean_docs[0])

# Compare the first 100 characters of the original and processed documents
with open(corpus_file, "r", encoding="utf-8") as file:
    original_docs = [line.strip() for line in file]

for i in range(min(len(original_docs), 5)):  # Display for the first 5 documents
    print(f"\nDocument {i + 1} (Original, First 100 chars):\n{original_docs[i][:100]}")
    print(f"Document {i + 1} (Processed, First 100 chars):\n{clean_docs[i][:100]}")
    

#### END YOUR CODE #####

Processing documents: 100%|██████████| 3916/3916 [06:55<00:00,  9.42doc/s]



Length of clean_docs =  3916
clean_docs[0]:
ninh thuận_địa_điểm ưu_tiên nhà_máy điện hạt_nhân góc ninh_thuận , địa_điểm ưu_tiên lựa_chọn nhà_máy điện hạt_nhân vương_hữu , viện trưởng viện năng_lượng nguyên_tử việt_nam , địa_điểm hiện cân_nhắc lựa_chọn nhà_máy điện hạt_nhân ninh_thuận , phú_yên phan_rang tuy_nhiên , ninh_thuận địa_điểm ưu_tiên lựa_chọn tiến_triển dự_án xây_dựng nhà_máy điện hạt_nhân vn , vương_hữu , viện năng_lượng nguyên_tử việt_nam hoàn_tất dự_án tiền_khả_thi trình chính_phủ phê_duyệt dự_kiến , ( 2007 ) , dự_án khả_thi đáp_ứng kịp tiến_độ thời_gian xây_dựng nhà_máy điện hạt_nhân vn kế_hoạch , nhà_máy điện hạt_nhân đầu_tiên xây_dựng đi hoạt_động bắt_đầu 2020 quy_mô công_suất 2 000 mw 4 000 mw , chiếm 5 9 tổng_công_suất phát_điện toàn_quốc vấn_đề lựa_chọn công_nghệ nhà_máy điện hạt_nhân vn , viện năng_lượng nguyên_tử việt_nam , hiện quá_trình phân_tích trưng_cầu ý_kiến chuyên_gia tuy_nhiên , công_nghệ nhà_máy điện hạt_nhân chọn_lựa cơ_sở đảm_bảo an_toàn quá_trình vận_h

## Question 4: Save Preprocessed Data
Hint: Save the preprocessed data to a file named `dataset_name + '.clean.txt'`, where each document is written on a separate line.


In [12]:
#### YOUR CODE HERE ####
output_file = f"{dataset_name}.clean.txt"
with open(output_file, "w", encoding="utf-8") as out_file:
    for doc in clean_docs:
        out_file.write(doc + "\n")

#### YOUR CODE HERE ####